In [1]:
from __future__ import print_function, division

In [2]:
from numpy import *

In [3]:
import scipy.spatial

In [4]:
## Generate a random affine matrix
def gen_random_affine():
    return random.random( (3,4) )

## Generate an affine identity matrix
def gen_identity_affine():
    return identity(4)[:3]

In [5]:
## Let's create some poses. The first (0-th) pose will be the identity.
num_handles = 2
num_poses = 3

poses_handles = []

poses_handles.append( [ gen_identity_affine() for i in range(num_handles) ] )
for pose in range( 1, num_poses ):
    poses_handles.append( [ gen_random_affine() for i in range(num_handles) ] )

## The dimensions of poses_handles are: #handles by #poses by affine 3x4 matrix
poses_handles = asarray( poses_handles )
poses_handles = swapaxes( poses_handles, 0, 1 )
print( poses_handles.shape )

(2, 3, 3, 4)


In [6]:
## Let's create some per-vertex skinning weights. Each vertex has a weight for each handle.
num_vertices = 100
## Let's assume there are two handles and the weights will vary linearly from 0 to 1 (and vice versa).
assert num_handles == 2
weights = zeros( ( num_vertices, num_handles ) )
weights[:,0] = linspace( 0, 1, num_vertices )
weights[:,1] = 1-weights[:,0]

## The dimensions of weights are #vertices by #handles.

## The weights must sum to one. Otherwise, we should normalize them once.
assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()

In [7]:
## The lbs() function generates the weighted average transformation.

## For all poses at once.
def lbs_all_poses( poses_handles, weights ):
    ## The weights across handles must sum to one.
    assert abs( weights.sum( axis = 1 ) - 1. < 1e-7 ).all()
    
    return ( weights[:,:,newaxis,newaxis,newaxis] * poses_handles[newaxis,...] ).sum( axis = 1 )

## For one set of handles (e.g. one pose) at a time.
def lbs_one_pose( handles, weights ):
    return lbs_all_poses( handles[:,newaxis,...], weights )

In [8]:
## lbs_one_pose() on pose 0 should give us an identity matrix for every vertex.
lbs_one_pose( poses_handles[:,0], weights )

array([[[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       ..., 
       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]],


       [[[ 1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.],
         [ 0.,  0.,  1.,  0.]]]])

In [9]:
pose = lbs_one_pose( poses_handles[:,2], weights )
# hull = scipy.spatial.ConvexHull( pose.reshape( num_vertices, -1 ) )

In [10]:
poses = lbs_all_poses( poses_handles, weights )
# hull = scipy.spatial.ConvexHull( poses.reshape( num_vertices, -1 ) )

In [11]:
## What are the dimensions of our poses? It should be #verts by #poses by 3x4 affine matrix.
print( poses.shape )
## Flattened it should be #vertices by (#poses*3*4)
X = poses.reshape( num_vertices, num_poses * 3 * 4 )
print( X.shape )

(100, 3, 3, 4)
(100, 36)


In [12]:
## Use SVD to get the lower dimensional space.
## The number of non-zero singular values is the number of handles for perfect reconstruction.
U, s, V = linalg.svd( X-average(X,axis=0)[newaxis,:], full_matrices = False, compute_uv = True )

In [13]:
## What are the singular values?
s.round(4)

array([ 5.9518,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

In [14]:
## Find the first index less than a threshold:
len(s)-searchsorted( s[::-1], 1e-6 )

1

In [15]:
## SVD reconstructs the original data
abs( X - (( U @ diag(s) @ V )+average(X,axis=0)[newaxis,:]) ).max()

6.6613381477509392e-16

In [16]:
## Formalize the above with functions.
def uncorrellated_pose_space( poses, threshold = 1e-6 ):
    X = poses.reshape( num_vertices, num_poses * 3 * 4 )
    
    ## Subtract the average.
    Xavg = average( X, axis = 0 )[newaxis,:]
    Xp = X - Xavg
    
    U, s, V = linalg.svd( Xp, full_matrices = False, compute_uv = True )
    
    ## The first index less than threshold
    stop_s = len(s) - searchsorted( s[::-1], threshold )
    
    def restore( uncorrellated_poses ):
        return ( ( uncorrellated_poses @ V[:stop_s] ) + Xavg ).reshape( -1, num_poses, 3, 4 )
    
    return Xp @ V[:stop_s].T, restore

In [17]:
## Vertify that we can go back and forth with these functions
uncorrelated_poses, restore = uncorrellated_pose_space( poses )
print( uncorrelated_poses.shape )
abs( restore( uncorrelated_poses ) - poses ).max()

(100, 1)


5.5511151231257827e-16

In [18]:
## Now let's use the convex hull on the uncorrelated poses.
# hull = scipy.spatial.ConvexHull( uncorrelated_poses )

In [19]:
uncorrelated_poses

array([[-1.02061681],
       [-0.99999829],
       [-0.97937977],
       [-0.95876125],
       [-0.93814273],
       [-0.9175242 ],
       [-0.89690568],
       [-0.87628716],
       [-0.85566864],
       [-0.83505012],
       [-0.8144316 ],
       [-0.79381308],
       [-0.77319455],
       [-0.75257603],
       [-0.73195751],
       [-0.71133899],
       [-0.69072047],
       [-0.67010195],
       [-0.64948343],
       [-0.6288649 ],
       [-0.60824638],
       [-0.58762786],
       [-0.56700934],
       [-0.54639082],
       [-0.5257723 ],
       [-0.50515378],
       [-0.48453525],
       [-0.46391673],
       [-0.44329821],
       [-0.42267969],
       [-0.40206117],
       [-0.38144265],
       [-0.36082413],
       [-0.3402056 ],
       [-0.31958708],
       [-0.29896856],
       [-0.27835004],
       [-0.25773152],
       [-0.237113  ],
       [-0.21649448],
       [-0.19587595],
       [-0.17525743],
       [-0.15463891],
       [-0.13402039],
       [-0.11340187],
       [-0

In [20]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
sns.regplot( x = uncorrelated_poses[:,0], y = uncorrelated_poses[:,1], fit_reg=False )
# plt.scatter( x = uncorrelated_poses[:,0], y = uncorrelated_poses[:,1] )

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
pmin, pmax = uncorrelated_poses.argmin(), uncorrelated_poses.argmax()
pmin, pmax

In [ ]:
poses_handles_restored = restore( array( [ uncorrelated_poses[ pmin ], uncorrelated_poses[ pmax ] ] ) )
print( poses_handles_restored.shape )
poses_handles_restored

In [ ]:
print( poses_handles.shape )
poses_handles

In [ ]:
abs( poses_handles_restored[0] - poses_handles[1] ).max()

In [ ]:
abs( poses_handles_restored[1] - poses_handles[0] ).max()